In [7]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import os.path

In [1]:
from wrangle_xml import wrangle_xml, preprocessing_xml

In [2]:
df = wrangle_xml()

In [3]:
df.head(2)

,creationDate,device,endDate,sourceName,sourceVersion,startDate,type,unit,value
0,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52 -0500,Noah's iPhone,9.3.1,2016-06-03 16:40:09 -0500,HKQuantityTypeIdentifierStepCount,count,70.0
1,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26 -0500,Noah's iPhone,9.3.1,2016-06-03 16:57:23 -0500,HKQuantityTypeIdentifierStepCount,count,68.0


In [4]:
df = preprocessing_xml(df)

In [5]:
df.head(2)

,creationDate,device,endDate,sourceName,sourceVersion,type,unit,value
startDate,,,,,,,,
2016-06-03 16:40:09,2016-06-03 17:38:46,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52,Noah's iPhone,9.3.1,HKQuantityTypeIdentifierStepCount,count,70.0
2016-06-03 16:57:23,2016-06-03 17:38:46,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26,Noah's iPhone,9.3.1,HKQuantityTypeIdentifierStepCount,count,68.0


In [8]:
df_2 = pd.read_csv("Export.csv")

In [9]:
df_2['Date'] = pd.to_datetime(df_2.Date)

In [10]:
df_2.Date.agg([min, max])

min   2018-08-26
max   2020-08-16
Name: Date, dtype: datetime64[ns]

In [14]:
df_2[df_2.Date('Y') == 2018]

TypeError: 'Series' object is not callable

In [12]:
df.type.value_counts()

HKQuantityTypeIdentifierStepCount                 67915
HKQuantityTypeIdentifierDistanceWalkingRunning    67908
HKQuantityTypeIdentifierFlightsClimbed             1201
Name: type, dtype: int64

In [ ]:
df = pd.read_csv("xml.csv")

In [ ]:
df.type.value_counts()

In [ ]:
df_steps = df[df.type == 'HKQuantityTypeIdentifierStepCount']

In [ ]:
df_steps

In [ ]:
df_steps = df_steps[['creationDate', 'endDate', 'startDate', 'value']]

In [ ]:
df_start = pd.DataFrame(df_steps.set_index('startDate').resample('d').value.sum())

In [ ]:
df_start